In [ ]:
# Data Management Project
# Team 6

In [ ]:
import pymongo # for MongoDB
import psycopg2 # for PostgresQL
import pandas as pd 

In [ ]:
import re
 
def get_text_sorted(text):

  res = [re.sub(r'[^\w\s]', '', s) for s in list(set(text.lower().split()))]
  res.sort()
  return res

def get_hashtags_sorted(hlist):
  hlist.sort()
  return hlist

In [ ]:
import json
import pymongo
from google.colab import drive
import time
import datetime
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Storing the Data

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/") 
db = client["myDB"]
tweets=db["tweets_table"]

In [ ]:
tweets.create_index([('created_at', pymongo.DESCENDING)])
tweets.create_index([('user_id', pymongo.ASCENDING)])
tweets.create_index([('id', pymongo.DESCENDING)])

ServerSelectionTimeoutError: ignored

In [ ]:

tweets.create_index([('text', 'text')])
def search_by_text_index(word):
  return tweets.find({'$text': {'$search':word}})

In [ ]:
def get_JSON_date(s):
  d=s.split()
  t=d[3].split(':')
  months={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
  m=datetime.datetime(day=int(d[2]), month=months[d[1]], hour=int(t[0]), minute=int(t[1]), second=int(t[2]), microsecond=int(d[4]), year=int(d[5]))

  return m.isoformat()

In [ ]:
# from Jinwoo

# connecting to the database
conn = psycopg2.connect(
    host = "localhost", #localhost
    port = "5432", #yourport
    database = "postgres", # databasename
    user="postgres",
    password = "Vicis9423*"
)
cur = conn.cursor() # make  cursor 


# creating user and hashtage table
cur.execute("ROLLBACK")
cur.execute("CREATE TABLE user_db (screen_name VARCHAR (100) PRIMARY KEY, user_id int,followers_count int, tweet_ids integer[]);")

NameError: ignored

In [ ]:
# Function for Inserting values to user_db and hashtag_db (PostgreSQL)

def insert_to_userdb(screen_name,user_id,followers_count,tweet_id):
    """ insert values into the user table """
    
    sql = """INSERT INTO user_db(screen_name, user_id ,followers_count, tweet_ids) VALUES(%s,%s,%s,%s);"""
    try:
        # execute the INSERT statement
        cur.execute("ROLLBACK")
        cur.execute(sql, (screen_name,user_id,followers_count,tweet_id))
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def update_tweet_ids_userdb(screen_name_,tweet_id):
  """ update values into the userdb table """

  sql =   """
  UPDATE user_db1 SET tweet_ids = array_append((SELECT tweet_ids FROM user_db1 WHERE screen_name=(%s)),(%s)) WHERE screen_name = (%s)  
  """
  try:
      # execute the UPDATE statement
      cur.execute("ROLLBACK")
      cur.execute(sql, (screen_name_,tweet_id,screen_name_))
  except (Exception, psycopg2.DatabaseError) as error:
      print(error) 



In [ ]:
# get the userdb
cur.execute("ROLLBACK")
userdb = cur.execute("""SELECT * FROM userdb""")

In [ ]:
#with open("corona-out-2", "r") as f1:
with open('/content/gdrive/My Drive/corona-out-2', "r") as f1:
    for line in f1:
        try:
          data = json.loads(line)
          myquery = { 'id': data['id'] }
          mydoc = tweets.find(myquery)



          # get the userdb
          cur.execute("ROLLBACK")
          user_db =  pd.read_sql_query('select * from "user_db"',con=conn) # dataframe format


            # pseudocode: 
            # if tweet has been seen before, 
            #         continue (ignore this tweet, go to next iteration of loop)
          if len(list(mydoc))==0:
            user = data['user']
            #add_user(user)
            # psuedocode: 
            # if user has not been seen before
            #            add to user table 
            # update datastores with any metrics that you are tracking 
            
            if ( data['text'].startswith('RT') ):
              retweet_data={'id':data['id'], 'created_at':get_JSON_date(data['created_at']), 'user_id':data['user']['id']}
              tdata=data['retweeted_status']
              myquery={'id':tdata['id']}
              original_tweet = tweets.find(myquery)
              if len(list(original_tweet))==0:
                tweets.insert_one({'id':tdata['id'], 'text':tdata['text'], 'text_sorted':get_text_sorted(data['text']), 'created_at':get_JSON_date(tdata['created_at']), 'user_id':tdata['user']['id'], 'favorite_count':tdata['favorite_count'], 'retweet_count':tdata['retweet_count'], 'interactions':tdata['favorite_count']+tdata['retweet_count'], 'hashtags':get_hashtags_sorted([x['text'].lower() for x in tdata['entities']['hashtags']]), 'retweets':[retweet_data]})
              
                # USER DB
           #     if tdata['user']['id'] in user_db:
           #       if tdata['id'] not in get_user_db_tweetlist(tdata['id'],'tweet_list'):
          #          update_tweet_list_userdb(tdata['user']'id'],tdata['id'])
            #    else:
             #     insert_to_userdb(tdata['user']['id'], tdata['user']['name'], tdata['favorite_count'],tdata['id'])


                # USER DB
                if tdata['user']['screen_name'] in list(user_db['screen_name']):
                  if tdata['id'] not in list(user_db['tweet_ids']):
                    update_tweet_ids_userdb(tdata['user']['screen_name'],tdata['id'])
                  else:
                    insert_to_userdb(tdata['user']['screen_name'], tdata['user']['id'], tdata['favorite_count'], [tdata['id']] )


              else:
               # retweets=original_tweet['retweets']
               # retweets.append(retweet_data)
                tweets.update_one(myquery, {'$push': {'retweets': retweet_data}})
                # psuedocode:
                # update retweet information
                # note that you may not have an entry for the original tweet 
                # if that is not in the dataset, put it in

            else:
              
              tweets.insert_one({'id':data['id'], 'text':data['text'], 'text_sorted':get_text_sorted(data['text']), 'created_at':get_JSON_date(data['created_at']), 'user_id':data['user']['id'], 'favorite_count':data['favorite_count'], 'retweet_count':data['retweet_count'], 'interactions':data['favorite_count']+data['retweet_count'], 'hashtags':get_hashtags_sorted([x['text'].lower() for x in data['entities']['hashtags']]), 'retweets':list()})


            # USER DB
            if data['user']['screen_name'] in list(user_db['screen_name']):
              if data['id'] not in list(user_db['tweet_ids']):
                update_tweet_ids_userdb(data['user']['screen_name'],data['id'])
            else:
                insert_to_userdb(data['user']['screen_name'], data['user']['id'], data['favorite_count'], [data['id']] )

                
        except:
          continue



In [ ]:

def binary_search(arr, low, high, s):
 
    if high >= low:
 
        mid = (high + low) // 2
 
        if arr[mid] == s:
            return True
 
        elif arr[mid] > s:
            return binary_search(arr, low, mid - 1, s)
 
        else:
            return binary_search(arr, mid + 1, high, s)
 
    else:
        return False

def search_by_text_or_ht(s, para):
  #para can be hashtags or text_sorted
  ret=list()
  for doc in tweets.find().sort('interactions', -1):
    if binary_search(doc[para], 0, len(doc)-1, s):
      ret.append(doc)


def time_range_search(t1, t2):
  #query={'$and':{[{'created_at':{'$gt':t1}}, {'created_at':{'$lt':t2}}]}}
  query={'created_at':{'$gt':t1}, 'created_at':{'$lt':t2}}
  return tweets.find(query).sort('interactions', -1)


def top_10_tweets():
  return tweets.find().sort('interactions', -1).limit(10)


# need to be optimzied 
def get_tweet_id_by_user_name(user_name):
  user_db = pd.read_sql("SELECT * FROM user_db",con=conn) # **** need to optimize
  return list(user_db[user_db['screen_name'] == user_name]['tweet_ids'])



def search_by_user(user_name):
  tweet_ids = get_tweet_id_by_user_name(user_name)
#want a get_user_name_by_user_id for meta data
  res=list()
  for tweet_id in tweet_ids:
      query={'id': tweet_id}
      for doc in tweets.find(query):
        res.append(doc)
  
  

## Relational

##NoSQL

Cache

In [ ]:
import redis
import json
from bson.json_util import dumps
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
redisClient = redis.StrictRedis(host='localhost', port=6379, db=0)

database = client['DummyDb'] #database name in mongodb
userList = database['Users'].find() #collection name in 'DummyDb' database

serializedObj = dumps(userList) #serialize object for the set redis.
result = redisClient.set('users', serializedObj) #set serialized object to redis server.

#you can check the users in redis using redis gui or type 'get users' to redis client or just get it from the redis like below.
parsedUserList = json.loads(redisClient.get('users'))

for user in parsedUserList: #check the names
	print(user["username"]) #'username' one of the field of the 'Users' collection

In [ ]:
import sqlite
import redis
import json

def get_tweet_data():
  connection = sqlite3.connect(database = "database.db")
  cursor = connection.cursor()
  redis_client = redis.Redis()

  cached_tweets = redis_client.get("tweet data") # Name of Key in cache
  if cached_tweet is not None:
    return json.load(cached_tweet)

  curson.execute("SELECT id, name FROM users;")
  result = cursor.fetchall()

  redis_client.set(name = "tweet data", value = json.dump(result), ex=60)  # Key = "tweet data",  Value = "result as json format" , ex mean expiration time

  cursor.close()
  connection.close()
  redis_client.close()
  return result


if __name__ == '__main__':
  print(get_tweet_data())


  

In [ ]:
# Cache

# Sample
import redis
import pymysql
import time

# 레디스 클라이언트를 만든다
# Redis<ConnectionPool<Connection>>
r = redis.Redis(host="192.168.88.244", port=6379, password="password", 
                decode_responses=True)
                
#pymysql로 mysql 연결
mysql_db = pymysql.connect(
    user="root",
    passwd="password",
    host="192.168.88.244",
    port=7676,
    db="employees",
    charset = "utf8"
)

#여기부터 실제 로직
result_list = []
#id=int(input("emp_no 입력 (6자리)")) #직접입력귀찮아서;
for emp_no in range(10200,10300):
    id= int(emp_no)
    first_time = time.time()
    result = r.get(name=id)
    print(id)

    if not result:
        print("캐시에서 가져오지 못함")
        cursor = mysql_db.cursor()
        sql = '''select emp_no,first_name from employees where emp_no=%s'''
        cursor.execute(sql, (int(id)))
        result = cursor.fetchone()
        r.set(name=result[0], value=result[1])


    else:
        print("캐시에서 가져옴!")

    result = r.get(name=id) #if문에 넣을수있지만 동일한 조건을 주기위함
    last_time = time.time()
    timetotime = last_time-first_time
    print(timetotime, "초 걸림")
    print(result)
    result_list.append(timetotime)
    
print("완료")
print(sum(result_list)/len(result_list)

# Search

In [ ]:
#based on the script above
#binary search
#how are we planning to store retweet data into MongoDB
#seach text,hashtag together or seperate
#DB connect when?
#search by text needs to be optimized depenging performance

tweets.create_index("hashtags")
tweets.create_index([('text', pymongo.ASCENDING)])

# Create hashtag to tweet_id to hash table
hash_tag_tweet_id = {}
for tweet in tweets:
    hast_tags = tweet.get('hashtags')
    for hash_tag in hast_tags:
        if hash_tag_tweet_id[hast_tag].count() == 0:
            hash_tag_tweet_id[hast_tag] = [tweet.get('id')]
        else:
            hash_tag_tweet_id[hash_tag].append(tweet.get('id'))
            
# Create interaction field in tweet mongoDB
def calculate_interaction():
    def transform(doc):
        retweet_count = len(tweet['retweets'])
        favorite_count = int(tweet.get('favorites'))
        doc['interactions'] = retweet_count + favorite_count
    return transform

tweet.update(calculate_interaction())
tweets.create_index([('interactions', pymongo.DESCENDING)])


def binary_search(arr, low, high, s):
 
    if high >= low:
 
        mid = (high + low) // 2
 
        if arr[mid] == s:
            return True
 
        elif arr[mid] > s:
            return binary_search(arr, low, mid - 1, s)
 
        else:
            return binary_search(arr, mid + 1, high, s)
 
    else:
        return False
    
def search_by_text_or_ht(s, para):
  #para can be hashtags or text_sorted
  ret=list()
  for doc in tweets.find().sort('interactions', -1):
    if binary_search(doc[para], 0, len(doc)-1, s):
      ret.append(doc)
    
# Find tweets by list of ids
def get_tweet_by_ids(tweet_ids):
    return tweets.find({'id':{$in:tweet_ids}}).sort('interactions', -1)

# Search directly into text field in tweet db using case-insensitve regex match
# Text field is indexed
# This might need optimization(still in progress)
def search_by_text(text):
    text = '/' + str(text) + '/i'
    return tweets.find({"text": re.compile(text, re.IGNORECASE)}).sort('interactions', -1)

# Search of hashtag key in hash_tag_tweet_id table
# After finding the value, list of tweet ids, for the hashtag
# find the tweets for those tweet ids
def search_by_hashtag(ht):
    if hash_tag_tweet_id.has_key(ht):
        tweet_ids =  hash_tag_tweet_id[ht]
        return get_tweet_by_ids(tweet_ids)
    else:
        return 'No Tweets for this hashtag'
            
def time_range_search(t1, t2):
    #query={'$and':{[{'created_at':{'$gt':t1}}, {'created_at':{'$lt':t2}}]}}
    query={'created_at':{'$gt':t1}, 'created_at':{'$lt':t2}}
    return tweets.find(query).sort('interactions', -1)


def top_10_tweets():
    return tweets.find().sort('interactions', -1).limit(10)

# Top 10 users orderer by follower counts 
# Direct postgreSQL Query into user_db 
def top_10_users():
    top_10_users =  pd.read_sql_query('select * from "user_db" ORDER BY followers_count DESC LIMIT 10',con=conn)
    return top_10_users

# need to be optimzied 
def get_tweet_id_by_user_name(user_name):
    user_db = pd.read_sql("SELECT * FROM user_db WHERE ",con=conn) # **** need to optimize
    return list(user_db[user_db['screen_name'] == user_name]['tweet_ids'])


def search_by_user(user_name):
    tweet_ids = get_tweet_id_by_user_name(user_name)
    #want a get_user_name_by_user_id for meta data
    return get_tweet_by_ids(tweet_ids)
  
  

# JINWOO"S EXPERIMENT

In [ ]:
with open('corona-out-2', "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            myquery = { 'id': data['id'] }
            mydoc = tweets.find(myquery)
            tweet_data = data
         
            # storing to USER DB in PostgreSQL
            ##########################################################
            user_screen_names = get_user_screen_names()
            user_tweet_ids = get_user_tweet_ids(tweet_data['user']['screen_name'])
            if tweet_data['user']['screen_name'] in user_screen_names:
                if tweet_data['id'] not in user_tweet_ids:
                    update_tweet_ids_userdb(tweet_data['user']['screen_name'],tweet_data['id'])
            else:
                insert_to_userdb(tweet_data['user']['screen_name'], tweet_data['user']['id'], tweet_data['user']['followers_count'], [tweet_data['id']] )
            ##########################################################

            
            # storing the tweeter data to MongoDB
            if len(list(mydoc))==0:
                user = data['user']
                if ( data['text'].startswith('RT') ):
                    retweet_data={'id':data['id'], 'created_at':get_JSON_date(data['created_at']), 'user_id':data['user']['id']}
                    tdata=data['retweeted_status']
                    myquery={'id':tdata['id']}
                    original_tweet = tweets.find(myquery)
                    if len(list(original_tweet))==0:
                    # storing to USER DB in PostgreSQL
                      ##########################################################
                      user_screen_names = get_user_screen_names()
                      user_tweet_ids = get_user_tweet_ids(tweet_data['user']['screen_name'])
                      if tweet_data['user']['screen_name'] in user_screen_names:
                        if tweet_data['id'] not in user_tweet_ids:
                          update_tweet_ids_userdb(tweet_data['user']['screen_name'],tweet_data['id'])
                        else:
                          insert_to_userdb(tweet_data['user']['screen_name'], tweet_data['user']['id'], tweet_data['user']['followers_count'], [tweet_data['id']] )
                     ##########################################################

                        tweets.insert_one({'id':tdata['id'], 'text':tdata['text'], 'text_sorted':get_text_sorted(data['text']), 'created_at':get_JSON_date(tdata['created_at']), 'user_id':tdata['user']['id'], 'favorite_count':tdata['favorite_count'], 'retweet_count':tdata['retweet_count'], 'interactions':tdata['favorite_count']+tdata['retweet_count'], 'hashtags':get_hashtags_sorted([x['text'].lower() for x in tdata['entities']['hashtags']]), 'retweets':[retweet_data]})
                    else:
                        tweets.update_one(myquery, {'$push': {'retweets': retweet_data}})
                else:
                    tweets.insert_one({'id':data['id'], 'text':data['text'], 'text_sorted':get_text_sorted(data['text']), 'created_at':get_JSON_date(data['created_at']), 'user_id':data['user']['id'], 'favorite_count':data['favorite_count'], 'retweet_count':data['retweet_count'], 'interactions':data['favorite_count']+data['retweet_count'], 'hashtags':get_hashtags_sorted([x['text'].lower() for x in data['entities']['hashtags']]), 'retweets':list()})
        except:
            continue

    